# Pandas and Matplotlib

This tutorial shows examples of using the `pandas` and `matplotlib` packages together. Data shaping tasks can be effectively done using `pandas` data frames, and visualizations can be created using `matplotlib`. Tasks in this tutorial may require either or both of these packages.

Basic knowledge of the `numpy`, `pandas`, and `matplotlib` packages is required.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Some exercises in this notebook (especially data shaping tasks) **may be mandatory** before continuing with subsequent tasks.

The Curry Shots dataset is used for this demonstration. This contains several thousand rows, each representing an attempted shot of Stephen Curry basketball player during his professional career.

If you run this notebook in Google Colab, do not forget to upload the file `curryshots.csv` for the session.

In [ ]:
df = pd.read_csv('curryshots.csv')
df.head(3)

Note: during this tutorial, it is often advisable to create a backup of a `DataFrame` in a given state, that can be restored if the data shaping steps need to be redone.

The `copy()` function creates a copy explicitly. Note that `df_orig = df` would only create a reference to the original.

In [ ]:
df_original_loaded = df.copy() # creating a backup

In [ ]:
df = df_original_loaded.copy() # loading the backup
df.head(3)

The starting point of most exercises and tasks will be a backup of the current state. Creating an own copy of this backup should be the first step of the solution code. This makes sure that the same cell can be executed multiple times, giving the same result. The same applies to the solution code.

# Data shaping

Many `pandas` functions have an option to pass `inplace=True` to mutate the current data frame instead of returning the result of the operation.

The following example shows how to rename and drop columns.

In [ ]:
df = df_original_loaded.copy()

df.rename(columns={'ID': 'row_id'}, inplace=True)
df = df.rename(columns={'Player': 'player_name'})
df.drop(['Season'], axis=1, inplace=True)
df.head(3)

Additional columns can be introduced using formulas based on other columns.

In [ ]:
df['Shot_Distance.m'] = df['Shot_Distance.ft.'] * 0.3048
df[['Shot_Distance.ft.', 'Shot_Distance.m']].head(3)

 The `loc` indexer with masks, the `map()` function, or `np.map()` can all be useful to define conditional columns.

In [ ]:
df.loc[df['Outcome (1 if made, 0 otherwise)'] == 1, 'Make or Miss'] = 'Make'
df.loc[df['Outcome (1 if made, 0 otherwise)'] == 0, 'Make or Miss'] = 'Miss'

In [ ]:
df['Make or Miss'] = df['Outcome (1 if made, 0 otherwise)'].map({1: 'Make', 0: 'Miss'})

In [ ]:
df['Make or Miss'] = np.where(df['Outcome (1 if made, 0 otherwise)'] == 1, 'Make', 'Miss')

In [ ]:
df[['Outcome (1 if made, 0 otherwise)', 'Make or Miss']].head(3)

If the definition of the new column is complex, we can use the `apply()` function. However, that iterates over all rows of the data frame, which is an anti-pattern, and can be slow for large datasets.

In [ ]:
def conv_time (quarter, clock):
  h, m = clock.split(':')
  clock_sec = int(h) * 60 + float(m)
  return 'Q' + str(quarter) + ', rem: ' + str(clock_sec) + ' s'

df['Shot_Time'] = df.apply(lambda row : conv_time(row['Quarter'], row['Game_Clock']), axis=1)
df[['Quarter', 'Game_Clock', 'Shot_Time']].head(5)

Vectorized functions are preferred for this tasks, especially if performance is critical. The following shows an equivalent definition using `str.split()`.

In [ ]:
h_m = df['Game_Clock'].str.split(':', expand=True).astype(float)
clock_sec = h_m[0]*60 + h_m[1]
df['Shot_Time'] = 'Q' + df['Quarter'].astype(str) + ', rem: ' + clock_sec.astype(str) + ' s'

df[['Quarter', 'Game_Clock', 'Shot_Time']].head(5)

Rearranging columns into a given order is as simple as listing the columns in the given order. With this approach, we can also perform a selection of columns.

In [ ]:
df_rearranged = df[['Date', 'Opponent', 'Location', 'Quarter', 'Outcome (1 if made, 0 otherwise)']]
df_rearranged.head(5)

However, this approach can become tedious if there are many columns. It is also possible to move a single column at once. The following example puts the `Shot_Time` column precisely before the `Date` column.

In [ ]:
column = df['Shot_Time']
df = df.drop(labels=['Shot_Time'], axis=1)
pos = df.columns.get_loc('Date')
df.insert(pos, 'Shot_Time', column)

df.head(3)

This can also be simplified into a one-liner as follows. `pop()` removes a columns and returns it.

In [ ]:
df.insert(df.columns.get_loc('Date'), 'Shot_Time', df.pop('Shot_Time'))
df.head(3)

### Exercise 1.

In [ ]:
df_starting_exercise_1 = df_original_loaded.copy()

* Rename a few columns to shorter names: `Outcome`, `PX` (left position), `PY` (top position), and `Distance` (of the shot).
* Remove the redundant rows `ID` and `Player`.
* Add a new column named `Scored` which is the `Shot_Value` if the `Outcome` is 1, and zero if the outcome is 0. Convert it into a numeric column.
* Add a new column named `YearMonth` which is the year and month value extracted from the `Date` column in YYYY-MM format. Arrange the new column directly after `Date`.

In [ ]:
df = df_starting_exercise_1.copy()

In [ ]:
to_rename = {
    'Outcome (1 if made, 0 otherwise)' : 'Outcome',
    'Left.px. (location)' : 'PX',
    'Top.px. (Location)' : 'PY',
    'Shot_Distance.ft.' : 'Distance'
}
# df = df.rename(columns=to_rename)
df.rename(columns=to_rename, inplace=True)
df

In [ ]:
df.drop(labels=['ID', 'Player'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df['Scored'] = df['Outcome'] * df['Shot_Value']
df[['Outcome', 'Shot_Value', 'Scored']]

In [ ]:
date_str = df['Date'].astype(str)
YY = date_str.str[-2:]
MM = date_str.str[:-4].str.rjust(2, '0')
df['YearMonth'] = '20' + YY + '-' + MM

df[['Date','YearMonth']]

In [ ]:
df.insert(df.columns.get_loc('Date') + 1, 'YearMonth', df.pop('YearMonth'))
df.head(3)

In [ ]:
df_result = df.copy()

# Creating plots

Note: the following codes and tasks require that the data shaping exercise is done, and the result is saved to `df`.

In [ ]:
df.head(3)

We will display the number of shots in each season as a bar and a pie chart.

First, perform the calculation of occurrences in a given column.

In [ ]:
season_counts = df['Season'].value_counts()
#season_counts = df['Season'].value_counts().sort_index()
#season_counts = df['Season'].value_counts().sort_values()

season_counts

Then, the data can be manually plotted with Matplotlib.

The data structures used by `pandas` are usually array-like, so they can be passed as arguments directly.

In [ ]:
bars = season_counts.index
lengths = season_counts.values

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.bar(bars, lengths)
ax.set_xticks(bars)
plt.show()

Observe that unlike the for the bars, the order of the pie slices is affected by how the data is sorted.

When passing columns separately, be sure that the row order remains consistent.

In [ ]:
lengths_sorted = season_counts.sort_index()
bars_sorted = lengths_sorted.index

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
ax1.pie(lengths, labels=bars)
ax1.set_title("lengths in original order")
ax2.pie(lengths_sorted, labels=bars_sorted)
ax2.set_title("lengths sorted by index")
plt.show()

We can use the `groupby()` function to perform a grouping operation.

The result is a special object, which (in short) can be used to perform aggregations.

In [ ]:
df_seasons = df.groupby(['Season'])

In [ ]:
bars = df_seasons.groups.keys()
bars

An aggregation can be peformed on the result of the `groupby()` function as follows. The result is a `DataFrame` again.

In [ ]:
df_scored = df_seasons[['Scored']].sum()
df_scored

We can manually construct the plots with Matplotlib.

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.bar(bars, df_scored['Scored'])
ax.set_xticks(list(bars))
plt.show()

Alternatively, `pandas` has a built in plotting feature, as demonstrated below.

In [ ]:
df_scored.plot(kind="bar", y='Scored')
plt.show()

We can choose multiple columns to summarize and to visualize after grouping.

In [ ]:
df_seasons[['Scored','Shot_Value']].sum().plot.line(y=['Scored','Shot_Value'])
plt.show()

Consider the following problem. We need to display a chart which shows total successful shots against different opponents (as colors), in each season (as an axis).

We can group by both the `Season` and `Opponent` fields.

Note: use double brackets when choosing the `Outcome` field, as the result should be a `DataFrame`. If a single bracket is used instead, the result is a `Series`.

In [ ]:
df_success = df.groupby(['Season','Opponent'])
df_success_sums = df_success[['Outcome']].sum()
df_success_sums

Plotting this aggregation is, however, is not what we want.

In [ ]:
df_success_sums.plot(y='Outcome')
plt.show()

We can perform an operation called **unstacking**.

From the multi-dimensional index, a field can be moved to form column labels.

Note: unstacking is roughly equivalent to the pivoting operation in Power BI, Power Query Editor.

In [ ]:
# df_success_sums.unstack() # same result
df_success_sums.unstack(level='Opponent')

Any column in the index can be chosen.

In [ ]:
df_success_sums.unstack(level='Season').head(7)

As an alternative to `groupby()` + `unstack()`, the function `pivot_table()` can be used that almost does the same.

In [ ]:
df_success_sums_withpivot = df.pivot_table(
    values="Outcome",
    index="Season",
    columns="Opponent",
    aggfunc="sum",
    # fill_value=0, # optional, replaces NaN values
)
df_success_sums_withpivot

A key difference is that `Outcome` is no longer part of the `DataFrame`. Therefore, when plotting, it does not need to be referenced. This is demonstrated in the following examples.

The following code plots the required chart, although the legend is ugly.

In [ ]:
# df_success_sums.unstack(level='Opponent').plot(y='Outcome')
df_success_sums_withpivot.plot()
plt.show()

With the `ax` parameter we can define and format the figure and the axis in the usual way, after populating it from a `DataFrame`.

Colors loop over a set of 10 colors by default, although this could be customized by assigning a custom color map. Note that with this number of opponents, multiple charts could be better.

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
df_success_sums.unstack(level='Opponent').plot(y='Outcome', ax=ax)
# df_success_sums_withpivot.plot(ax=ax)
ax.legend(loc="upper center", ncol=6, bbox_to_anchor=(0.5,1.5))
plt.show()

Alternatively, a stacked bar chart is easy to draw using this method.

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
df_success_sums.unstack(level='Opponent').plot(
    kind="bar", y='Outcome', stacked=True, ax=ax)
# df_success_sums_withpivot.plot(kind="bar", stacked=True, ax=ax)
ax.legend(loc="upper center", ncol=6, bbox_to_anchor=(0.5,1.5))
plt.show()

Alternatively, we can extract and prepare the data manually, as shown below.

In [ ]:
df_success_sums = df_success[['Outcome']].sum()

# years = set()
# opponents = set()
# for year, opponent in df_success_sums.index:
#   years.add(year)
#   opponents.add(opponent)
# years = sorted(list(years))
# opponents = sorted(list(opponents))
years = sorted(df_success_sums.index.get_level_values(0).unique())
opponents = sorted(df_success_sums.index.get_level_values(1).unique())

fig, ax = plt.subplots(figsize=(7,4))

bar_bottoms = np.zeros(len(years))
for opponent in opponents:
  lengths = []
  for year in years:
    lengths.append(df_success_sums.loc[year,'Outcome'].get(opponent, 0))
  ax.bar(years, lengths, width=0.8, label=opponent, bottom=bar_bottoms)
  bar_bottoms += lengths

ax.set_xticks(years)
ax.legend(loc="upper center", ncol=6, bbox_to_anchor=(0.5, 1.5))
plt.show()

### Exercise 2.

Display the positions of all shots in the last season. Show successful and unsuccessful shots in different styles.

### Exercise 3.

Display the same positions on the scatter chart, but denote outcome by different markers instead of colors.

### Exercise 4.

Display a chart which shows total scored points based on distance groups 0-4, 5-12, 13-21, and 22+ feet (as an axis), and by location (home or away, as color).

Tip: check out the documentation of the `cut()` function in `pandas`.

### Exercise 5.

Plot the shot success rate for each season, further categorized by shot value.